# Data Cleaning

Real world data is usually messy and can contain errors. These can be in the form of unnecessary (sometimes empty) columns, values not formatted correctly or recorded erroneously.

In this notebook we present a few data related issues and techniques on how to tackle these issues. We will first use the `student_debt.csv` data as previously.

In [1]:
import pandas as pd

df = pd.read_csv("data/student_debt.csv")
df

,Unnamed: 0,Period,Characteristic,No.people,Sum,Average,Median
0,2,2011,Total,754.2,9.5,12.6,7.4
1,3,2011,Man,391.0,5.1,13.1,7.6
2,4,2011,Woman,363.1,4.4,12.0,7.2
3,5,2011,up to 20 years old,41.1,0.1,2.5,1.4
4,6,2011,between 20 and 25 years old,284.8,2.3,8.1,4.8
...,...,...,...,...,...,...,...
67,69,2019*,up to 20 years old,104.5,0.4,4.1,2.5
68,70,2019*,between 20 and 25 years old,479.0,5.2,10.9,7.2
69,71,2019*,between 25 and 45 years old,822.2,13.6,16.5,10.6
70,72,2019*,between 45 and 65 years old,7.7,0.1,12.4,6.2


Notice the column `Unnamed: 0` does not contain any useful information, it is a mismatched version of the index only. This happens often when the data is downloaded, created or otherwise transformed as a side effect. 

We can easily get rid of a column in Pandas with the `drop()` function. The drop function accepts a list of column names in the columns attribute. 
Furthermore, you can pass `inplace = True` to perform the operation on the dataframe. If you do not do this, the operation returns a new dataframe without the specified columns.

In [2]:
df_dropped_column = df.drop(columns=["Unnamed: 0"]) #returns a new dataframe 
df.drop(columns=["Unnamed: 0"], inplace=True)
## Note that now df_dropped_column and df is exactly the same

Now that we have only the useful columns, we can look at the values in our data set. 
For this a convenient starting point is looking at whether all data types are as expected. 

In [3]:
df.dtypes

Period             object
Characteristic     object
No.people         float64
Sum               float64
Average           float64
Median            float64
dtype: object

Notice that `Period` has a data type object, whereas it should be an integer or float. 
If you look at some of the year values, they have an asterisk * next to them. This is because those figures are preliminary only, but we still need those years in raw number format, such that we can perform analysis on it.

We can use the `.apply()` function. This applies a custom function on every row of the dataframe its called upon.
For simpler functions we can write a lambda function. Don't let the fancy name fool you, they are actually pretty easy.

In python we can call the `replace()` method on any string to replace a piece of string to something user specified. We can use this to get rid of the * after the years. Then we can convert the years to integers with `int()`.

In [4]:
# we specifiy a lambda function which applies a certain operation on every row of our dataframe
df.apply(lambda row:int(row['Period'].replace('*', '')), axis=1) 
#axis = 1 needed to apply the operations on every row, axis = 0 would apply the operations on every column


0     2011
1     2011
2     2011
3     2011
4     2011
      ... 
67    2019
68    2019
69    2019
70    2019
71    2019
Length: 72, dtype: int64

Notice that it returns a new Pandas series, and does not modify the original dataframe. We can create or replace a column with pandas by simply specifying the column name and the values of the new column.

In [5]:
df["Period"] = df.apply(lambda row:int(row['Period'].replace('*', '')), axis=1) 
df.dtypes

Period              int64
Characteristic     object
No.people         float64
Sum               float64
Average           float64
Median            float64
dtype: object

Notice that the data type of Period now changed to int64, which was our desired goal.

# Missing values and outliers on Salary data set

In some data sets you have missing or otherwise erroneous values. In the data dashboard there are only data sets without these issues, but in general, it is a very common and important step in data cleaning.

For this we will use a different data set downloaded from our data dashboard (can also be found in the included).

In [6]:
df_salaries = pd.read_csv("data/DenBosch-avg-salary.csv")
df_salaries.iloc[40:50]

,AreaCode,Neighbourhood code,Total avg. salary (thousand EUR per year),<25yrs old,25-44yrs old,45-64yrs old,65+ yrs old
40,06,Totaal,"35,4","11,1","42,3","46,3","25,6"
41,06,01,"33,7","10,9","44,1","43,0","24,8"
42,06,02,"32,0","12,9","42,3","40,4","24,9"
43,06,03,"34,8","10,8","41,3","46,2","24,9"
44,06,04,"38,5",.,.,"50,1",.
45,06,05,.,.,.,.,.
46,06,06,"36,0","9,3","40,8","46,3","26,6"
47,06,07,"38,7","11,6","42,6","50,2","28,4"
48,06,08,.,.,.,.,.
49,06,09,.,.,.,.,.


In [7]:
df_salaries.dtypes

AreaCode                                     object
Neighbourhood code                           object
Total avg. salary (thousand EUR per year)    object
<25yrs old                                   object
25-44yrs old                                 object
45-64yrs old                                 object
65+ yrs old                                  object
dtype: object

As you can see, part of this data has missing values (in the form of `.`) in several places. Furthermore also notice that all the numerical values are listed as objects, because Python uses a `.` to indicate decimals, and in this data set we have `,` separating them. 

First we have to do two things: replace the `,` to `.` in order to convert the numbers to `float`, and set the missing values to be actually missing, this latter can be done by filtering cells containing a `.` and setting their values to `np.nan`.

For this exercise, we select only the columns we want to clean. In our case, we only want columns with numbers, so we can drop *AreaCode, Neighbourhood code* columns for imputation.

Later on, we need to recombine (concatenate) the imputed columns with the other two we decided to drop.

In [8]:
import numpy as np 

columns_not_impute = ['AreaCode', 'Neighbourhood code'] # columns we don't want to preprocess further
df_salaries_to_impute = df_salaries.drop(columns=columns_not_impute)

#iterate over all columns and replace , with . using a lambda function
for column in df_salaries_to_impute.columns:
    df_salaries_to_impute[column] = df_salaries_to_impute.apply(lambda row:row[column].replace(',', '.'), axis=1)

df_salaries_to_impute[df_salaries_to_impute.values == '.'] = np.nan #set the missing values to NaN

df_salaries_to_impute = df_salaries_to_impute.astype(float)


Now with the missing values we can do something called *missing value imputation*, which replaces missing values with some plausible value.
For this we will use the Python library scikit-learn, that has multiple methods to impute missing values.
In this tutorial we use the `SimpleImputer`, which imputes the average value of each column with missingness.
NOTE: This imputation algorithm only accepts a dataframe that contains one specific data type. If we specify `SimpleImputer(strategy = 'mean')` we can impute numerical values only. However, if we specify `SimpleImputer(strategy = 'median')` we can fill in the missing values for textual fields.


Before that we can look at how many missing values we have for each column, this can be done by using `isna()` that returns true if a cell is missing and False otherwise, and then `.sum()` over these values to get the number of missing values per column. In the code cell below we also divided by the number of rows, such that the values can be interpreted as percentages.

In [9]:
# 
df_salaries_to_impute.isna().sum()/df_salaries_to_impute.shape[0]

Total avg. salary (thousand EUR per year)    0.204918
<25yrs old                                   0.409836
25-44yrs old                                 0.327869
45-64yrs old                                 0.319672
65+ yrs old                                  0.377049
dtype: float64

In [10]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean') #initialize scikit learn imputer
imputed_values = imputer.fit_transform(df_salaries_to_impute)
### You can see that the imputation algorithm returns values only.
### We can convert this into a DataFrame by doing:
df_imputed_salaries = pd.DataFrame(imputed_values, columns=df_salaries_to_impute.columns, index=df_salaries_to_impute.index)
# Here we specify that we want a new dataframe wiht the imputed values and the same columns and index as the previous data frame

df_imputed_salaries

,Total avg. salary (thousand EUR per year),<25yrs old,25-44yrs old,45-64yrs old,65+ yrs old
0,35.4,11.700000,41.700000,45.600000,26.100000
1,40.0,15.900000,42.200000,55.800000,31.300000
2,41.5,17.000000,42.500000,59.400000,34.200000
3,41.6,14.300000,43.000000,59.500000,34.100000
4,32.3,14.700000,40.000000,34.800000,21.500000
...,...,...,...,...,...
117,44.8,11.305556,42.186585,45.136145,25.494737
118,32.2,14.700000,43.100000,40.000000,21.500000
119,32.7,14.700000,42.400000,42.500000,21.800000
120,29.3,11.305556,42.186585,33.000000,22.200000


Now all we need to do is use `pd.concat()` to join the data with the discarded area codes.

In [11]:
df_salaries_cleaned = pd.concat([df_salaries[columns_not_impute], df_imputed_salaries], axis=1)
df_salaries_cleaned

,AreaCode,Neighbourhood code,Total avg. salary (thousand EUR per year),<25yrs old,25-44yrs old,45-64yrs old,65+ yrs old
0,Totaal,Totaal,35.4,11.700000,41.700000,45.600000,26.100000
1,01,Totaal,40.0,15.900000,42.200000,55.800000,31.300000
2,01,01,41.5,17.000000,42.500000,59.400000,34.200000
3,01,02,41.6,14.300000,43.000000,59.500000,34.100000
4,01,03,32.3,14.700000,40.000000,34.800000,21.500000
...,...,...,...,...,...,...,...
117,13,99,44.8,11.305556,42.186585,45.136145,25.494737
118,14,Totaal,32.2,14.700000,43.100000,40.000000,21.500000
119,14,01,32.7,14.700000,42.400000,42.500000,21.800000
120,14,02,29.3,11.305556,42.186585,33.000000,22.200000


Finally, we can save the cleaned dataframe to a csv file, so that we can access it later. 
For this we can call the `to_csv()` method of a dataframe.
It is important that you specify `index = False` in the arguments, as we don't want to save indices, those are auto-generated by Pandas every time we open a new dataframe.

In [12]:
df_salaries_cleaned.to_csv("data/cleaned-DenBosch-avg-salary.csv", index=False)

# Exercises

In these exercises you will perform similar cleaning tasks on several other data sets downloaded from our data dashboard.

The first data set we will use is the `municipality_data.csv`.

### Exercise 1

Load the data and check which parts need cleaning. Look for extra columns, mismatching data types, check the percentage of missing values. 


In [13]:
#------------ Exercise code comes here ------------- #


### Exercise 2

Using the methods presented above, try and fix those inconsistencies. If you need to do missing value imputation, use the `SimpleImputer` with the correctly specified strategy.
Remember the steps how to fill in missing values:
- Check by hand which columns _should_ have numerical values.
- Check data types with `.dtypes` whether in fact they are numerical.
    - If not identify the cause (letters next to the numbers, dots for missing values instead of NaN etc.)
    - Replace/remove text from all columns that should be numerical
- Separate only numerical columns into a new dataframe
- Perform the imputation using the `SimpleImputer` 
- Merge (concatenate) the imputed numerical values with the (excluded) non-numerical values to get the original dataframe back

If you are stuck anywhere along the process, feel free to reach out to one of the Data mentors.

In [14]:
# ------------ Exercise code comes here ------------- #

### Exercise 3

Save the cleaned dataframe to a new csv file.

In [15]:
# ------------ Exercise code comes here ------------- #

## Cleaning second data set

Below we have some other practice material on another data set named `Farmers_NH3_emission.csv`.

### Exercise 4
Check the data types of the column. You should see that some data types do not match because of the units in the cells next to the numbers. Try to get rid of them and convert the columns to numbers (float or int as necessary).

As practice you can also check if the data contains any missing values.

In [16]:
# ------------ Exercise code comes here ------------- #